<img src= "img/pipelines.png" style="height:450px">


[Image Source](https://towardsdatascience.com/using-functiontransformer-and-pipeline-in-sklearn-to-predict-chardonnay-ratings-9b13fdd6c6fd)

__Agenda__

- Pipelines and Composite estimators

- Why do we need them?

- How to use them in sklearn: accessing a particular object in pipe and changing parameters

- Combining pipelines with gridsearch

- Summary and further reading

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler, Normalizer


# Pipelines

__What is a Pipeline?__

_Transformers:_ Any object with .transform method. Ex: PCA, OneHotEncoder.

_Estimators:_ Any object with predict method. Ex: RandomForestClassifier, LinearRegression etc.

_Pipelines:_ A tool for combining transformers with estimators. 


__Other relevant tools are:__

- FutureUnion

- TransformedTargetRegressor

__Why do we need pipelines?__

- Convenience and encapsulation

Even though we train 10 transformers and 5 estimators we will call fit and predict once.

- Joint parameter selection - here emphasize preprocessing part

We can put pipelines into gridsearch and find best parameters for all the estimators at once.

- Safety

Pipelines help avoid leaking statistics from your test data into trained model.

[Pipelines and Composite Estimators](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)



## Usage of Pipelines

The Pipeline is built using a list of (key, value) pairs, where the key is a string containing the name you want to give this step and value is an estimator object:

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA 
from sklearn.impute import SimpleImputer

estimators = [('imputer', SimpleImputer()),('reduce_dim', PCA()), ('clf', SVC())]

pipe = Pipeline(estimators)

pipe



Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('clf',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [ ]:
estimators = [('imputer', SimpleImputer()),('reduce_dim', PCA()), ('clf', SVC())]

pipe = Pipeline(estimators)

pipe

__Your Turn__

- Create your own pipeline. You can use the same transformers and estimators with different parameters and 'name'.

- You can create a new pipe with a scaler also.

In [3]:
pipe = Pipeline([('imputer', SimpleImputer()),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(C=1000,
                                            max_iter=1000,
                                            solver='saga'))])

In [4]:
pipe.named_steps['imputer']

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [5]:
pipe.named_steps['scaler']

StandardScaler(copy=True, with_mean=True, with_std=True)

In [6]:
pipe.named_steps['clf']

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
pipe.steps

[('imputer', SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                missing_values=nan, strategy='mean', verbose=0)),
 ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('clf',
  LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=1000,
                     multi_class='warn', n_jobs=None, penalty='l2',
                     random_state=None, solver='saga', tol=0.0001, verbose=0,
                     warm_start=False))]

Sklearn also gives us "make_pipeline" which is almost the same thing but with make_pipeline you don't have to give names.

__Your Turn__

-  [Check documentation: 6.1.1.1.1. Construction](https://scikit-learn.org/stable/modules/compose.html) and use make_pipeline to construct an pipeline.

In [10]:
pipe = make_pipeline(SimpleImputer(), MinMaxScaler(), RandomForestClassifier(max_depth = 10))

In [11]:
pipe

Pipeline(memory=None,
         steps=[('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=10,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators='warn', n_jobs=None,
  

In [15]:
pipe.named_steps['scaler']

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:
from sklearn.pipeline import make_pipeline

In [12]:
# %load -r 1-10 supplement.py
pipe = Pipeline([('imputer', SimpleImputer()),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(C = 1000,
                                                max_iter = 1000,
                                                solver = 'saga'))])

## we can access to a particular step in the pipeline

pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)

NameError: name 'X_train' is not defined

## Accessing steps

We have multiple ways to access and object in the pipeline

- steps attribute

- [idx]



In [16]:
## note that these will all give the simple imputer object
pipe.steps[0]

pipe['simpleimputer']

pipe[0]

KeyError: 'simpleimputer'

In [17]:
## We can also access a particular object by named_steps
## sklearn claims that tab completion should work here but 
## in my notebook it didn't

pipe.named_steps.simpleimputer

AttributeError: simpleimputer

In [18]:
## We can 'slice' pipelines to create sub-pipes

pipe[1:]

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 LogisticRegression(C=1000, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

## Access to the parameters

Parameters of the estimators in the pipeline can be accessed using the 
"estimator__parameter" syntax.

In [19]:
pipe.set_params(clf__C = 10)

pipe

Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 LogisticRegression(C=10, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

## Caching Transformers

[(6.1.1.3. Caching transformers: avoid repeated computation)](https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators)

There are two ways of caching transformers:

1. Use mkdtemp

In [21]:
from sklearn.datasets import load_boston

X, y = load_boston(return_X_y=True)

In [22]:
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
estimators = [('reduce_dim', PCA()), ('clf', LinearRegression())]
cachedir = mkdtemp()
pipe = Pipeline(estimators, memory=cachedir)
pipe.fit(X, y)

Pipeline(memory='C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\tmpmc5q3hns',
         steps=[('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('clf',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [23]:
# Clear the cache directory when you don't need it anymore
rmtree(cachedir)

In [24]:
!ls

data
img
pipelines.ipynb
supplement.py


Or by giving a string as the directory names

In [ ]:
pipe = Pipeline(estimators, memory='cached_transformers')
pipe.fit(X, y)

In [ ]:
## again remove it when you are done with them
rmtree('cached_transformers')

## Transforming target in regression

In [25]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X, y = load_boston(return_X_y=True)
transformer = QuantileTransformer(output_distribution='normal')
regressor = LinearRegression()
regr = TransformedTargetRegressor(regressor=regressor,
                                  transformer=transformer)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
regr.fit(X_train, y_train)

print('R2 score: {0:.2f}'.format(regr.score(X_test, y_test)))

raw_target_regr = LinearRegression().fit(X_train, y_train)
print('R2 score: {0:.2f}'.format(raw_target_regr.score(X_test, y_test)))

R2 score: 0.67
R2 score: 0.64


C:\Users\Akshay Indusekar\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (379). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


## Using Pipelines with GridSearchCV

In [26]:
pipe

Pipeline(memory='C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\tmpmc5q3hns',
         steps=[('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('clf',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [27]:
# we can also access the parameters in the gridsearch

from sklearn.model_selection import GridSearchCV
param_grid = dict(simpleimputer__strategy=['mean', 'median'],
                  svc__C=[0.1, 10, 100])
grid_search = GridSearchCV(pipe, param_grid=param_grid)

grid_search

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory='C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\tmpmc5q3hns',
                                steps=[('reduce_dim',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('clf',
                                        LinearRegression(copy_X=True,
                                                         fit_intercept=True,
                                                         n_jobs=None,
                                                         normalize=False))],
                                verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'sim

In [31]:
# note also that we can choose to skip some steps in the pipeline

param_grid = dict(reduce_dim=['passthrough', PCA(5), PCA(6)],
                  svc=[SVC(gamma='auto'),
                       LogisticRegression(solver='lbfgs', max_iter=1000)],
                  svc__C=[0.1, 10, 100])

grid_search = GridSearchCV(pipe, param_grid=param_grid)

In [32]:
grid_search.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [33]:
grid_search.score(iris.data, iris.target)

NameError: name 'iris' is not defined

## Pipelines in action

In [34]:
df = pd.read_csv('data/diabetes.csv')
display(df.head(), df.shape)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


(768, 9)

In [35]:
target = df.Outcome
column_list = df.columns.tolist()

column_list.remove('Outcome')
print(column_list)

data = df[column_list]

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


[On Scaling data](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df,
                                                    target,
                                                    test_size=0.20,
                                                    stratify=target,
                                                    random_state=120919)

__Your Turn__

- Create a pipeline and use this pipeline for fitting and predicting diabetes results for the above data.


In [39]:
# %load -r 1-10 supplement.py
pipe = Pipeline([('imputer', SimpleImputer()),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(C = 1000,
                                                max_iter = 1000,
                                                solver = 'saga'))])

## we can access to a particular step in the pipeline

pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)

1.0

__Your Turn__

- Now use gridsearch with pipelines and return the best parameters

In [41]:
# %load -r 13-22 supplement.py

param_grid = dict(scaler=['passthrough',MinMaxScaler() , Normalizer(), StandardScaler()],
                  clf = [SVC(gamma = 'auto'),
                         LogisticRegression(solver = 'lbfgs',max_iter =1000)],
                  clf__C = [0.1, 10, 100])

gs = GridSearchCV(pipe, param_grid=param_grid, cv = 3, verbose = 1)

gs.fit(X_train, y_train)


Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    1.9s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('imputer',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
                                                      strategy='mean',
                                                      verbose=0)),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        LogisticRegression(C=1000,
                                                        

__Remark__


Note that even if gridsearch and pipes are getting along very well. The options are not limitless. Try to add Randomforests as classifier in the gridsearch. 

## Further research and miscellaneous

- [FeatureUnion](https://scikit-learn.org/stable/modules/compose.html#featureunion-composite-feature-spaces)

- [ColumnTransformer](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data)

- [sklearn, dictionary of terms](https://scikit-learn.org/stable/glossary.html#term-transformer)

- [Pydata meeting on pipelines](https://www.youtube.com/watch?v=BFaadIqWlAg)

- [Another pydata talk on pipelines with FeatureUnion](https://www.youtube.com/watch?v=URdnFlZnlaE)

- [On scalers](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler)

- [A nice notebook on pipelines](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb)

In [42]:
!pip install Shapely
!pip install Geopandas
!pip install rtree

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Akshay Indusekar\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\pip-install-867gzlxt\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\pip-install-867gzlxt\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: C:\Users\AKSHAY~1\AppData\Local\Temp\pip-install-867gzlxt\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Akshay Indusekar\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\pip-install-yjbs2okk\\rtree\\setup.py'"'"'; __file__='"'"'C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\pip-install-yjbs2okk\\rtree\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: C:\Users\AKSHAY~1\AppData\Local\Temp\pip-install-yjbs2okk\rtree\
    Complete output (11 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\AKSHAY~1\AppData\Local\Temp\pip-install-yjbs2okk\rtree\setup.py", line 3, in <module>
        import rtree
      File "C:\Users\AKSHAY~1\AppData\Local\Temp\pip-install-yjbs2okk\rtree\rtree\__init__.py", line 1, in <module>
        fro

In [ ]:
#conda env create -f gis-env .yml  in terminal window with the folder and run
#conda activate gis-env and then conda install rtree

In [43]:
!pip install Geopandas

  Using cached https://files.pythonhosted.org/packages/83/c5/3cf9cdc39a6f2552922f79915f36b45a95b71fd343cfc51170a5b6ddb6e8/geopandas-0.7.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6d/42/f4a7cac53b28fa70e9a93d0e89a24d33e14826dad6644b699362ad84dde0/Fiona-1.8.13.post1.tar.gz


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Akshay Indusekar\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\pip-install-bt5stu2d\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\AKSHAY~1\\AppData\\Local\\Temp\\pip-install-bt5stu2d\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: C:\Users\AKSHAY~1\AppData\Local\Temp\pip-install-bt5stu2d\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [44]:
%conda install geopandas


Note: you may need to restart the kernel to use updated packages.



EnvironmentLocationNotFound: Not a conda environment: C:\Users\Akshay

